## Techniche - Topic Modelling

In [14]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel, atmodel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import sys

from test_model import tokenize_docs, clean_docs, lower_words, remove_stopwords, get_topics#, (TODO) Lee convert_bytes

from smart_open import smart_open

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

import pyspark
import pyspark.sql.functions as F

In [15]:
%load_ext autoreload

# pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 10
pd.set_option('display.max_rows', 10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
np.random.seed(3)

In [34]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

### Import Data

#### Import data from PatentsView API

In [35]:
# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("/Users/lee/Documents/techniche/techniche/data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

#### Import initial dataset

In [36]:
# build query - initial small dataset

query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},
              {"_text_phrase":{"patent_abstract":"natural language"}}]}
# uncomment to use alternate query options
# query={"cpc_subgroup_id":"G06T3/4046"}
# query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
# query={"_and":
#         [{"_or":
#             [{"_text_phrase":{"patent_title":"machine learning"}}
#             ,{"_text_phrase":{"patent_abstract":"machine learning"}}]}
#         ,{"_and":
#       [{"patent_year":2016}]}]}
# query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},{"_text_phrase":{"patent_abstract":"natural language"}}]}
# uncomment to use alternate query options
# query={"cpc_subgroup_id":"G06T3/4046"}
# query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
# query={"_and":
#         [{"_or":
#             [{"_text_phrase":{"patent_title":"natural language"}}
#             ,{"_text_phrase":{"patent_abstract":"natural language"}}]}
#         ,{"_and":
#       [{"patent_year":2016}]}]} 
# query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
fields=pat_fields
options={"per_page":100}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
response = requests.get(endpoint_url, params=params)
status = response.status_code
print("status:", status)
results = response.json()
count = results.get("count")
total_pats = results.get("total_patent_count")
print("patents on current page:",count,';', "total patents:",total_pats)

status: 200
patents on current page: 100 ; total patents: 2482


#### Structure data

In [37]:
# extract metadata from response
print("status code:", response.status_code,';', "reason:", response.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data_resp = results['patents']
# data_resp[0]

raw_df = pd.DataFrame(data_resp)
raw_df.head(3)

status code: 200 ; reason: OK
total_patent_count: 2482 ; patents_per_page: 100


,IPCs,application_citations,applications,assignees,cited_patents,...,patent_year,pct_data,rawinventors,uspcs,wipos
0,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020077823', 'ap...","[{'app_country': 'US', 'app_date': '2013-07-26...","[{'assignee_city': 'Burlington', 'assignee_cou...",[{'cited_patent_category': 'cited by examiner'...,...,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Jeffrey N.', 'raw...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
1,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020138265', 'ap...","[{'app_country': 'US', 'app_date': '2017-09-11...","[{'assignee_city': 'Mountain View', 'assignee_...",[{'cited_patent_category': 'cited by applicant...,...,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Evgeny A.', 'rawi...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
2,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2001/20010029455', 'ap...","[{'app_country': 'US', 'app_date': '2016-09-28...","[{'assignee_city': 'Seattle', 'assignee_countr...",[{'cited_patent_category': 'cited by applicant...,...,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Thibault Pierre',...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."


#### Subset dataframe

In [38]:
# subset dataframe - comment/uncomment to include fields
df = raw_df[['patent_number', 
         'patent_date', 
         'patent_title',
         'patent_abstract', 
         'patent_firstnamed_assignee_id',
         'patent_firstnamed_assignee_location_id',
         'patent_firstnamed_assignee_latitude',
         'patent_firstnamed_assignee_longitude',
         'patent_firstnamed_assignee_city',
         'patent_firstnamed_assignee_state',
         'patent_firstnamed_assignee_country', 
         'patent_firstnamed_inventor_id',
         'patent_firstnamed_inventor_location_id',
         'patent_firstnamed_inventor_latitude',
         'patent_firstnamed_inventor_longitude',
         'patent_firstnamed_inventor_city',
         'patent_firstnamed_inventor_state',
         'patent_firstnamed_inventor_country',
         'patent_year', 
         'patent_type', 
         'patent_kind',
         'inventors'
            ]]
df.head(3)

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,...,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors
0,10229106,2019-03-12,Initializing a workspace for building a natura...,Designing a natural language understanding (NL...,org_ID497r4tFbCIaMBjGAST,...,US,2019,utility,B2,"[{'inventor_city': 'Newton', 'inventor_country..."
1,10229109,2019-03-12,Allowing spelling of arbitrary words,"Methods, systems, and apparatus, including com...",org_p6ofWD2xFNSnyYkj6wpA,...,CH,2019,utility,B1,"[{'inventor_city': 'Adliswil', 'inventor_count..."
2,10229113,2019-03-12,Leveraging content dimensions during the trans...,A content management system (CMS) and a transl...,org_Vbc6obpnxWM42d0HjlXY,...,US,2019,utility,B1,"[{'inventor_city': 'Seattle', 'inventor_countr..."


#### Explore data

In [39]:
# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

39

#### Create new column

In [40]:
# create new column that combines the patent title and the patent abstract columns into a single string
df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    Initializing a workspace for building a natura...
1    Allowing spelling of arbitrary words Methods, ...
2    Leveraging content dimensions during the trans...
Name: patent_title_abstract, dtype: object

In [41]:
df.sort_values(by=['patent_date'])

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,...,patent_year,patent_type,patent_kind,inventors,patent_title_abstract
99,10162315,2018-12-25,Process control system using typical and adapt...,"Methods, systems, and non-transitory, computer...",org_Y5Tht7WZIuEYKAJwDAZQ,...,2018,utility,B2,"[{'inventor_city': 'Tarentum', 'inventor_count...",Process control system using typical and adapt...
78,10168988,2019-01-01,Identifying user preferences and changing sett...,"A method, a computer program product, and a co...",org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Beijing', 'inventor_countr...",Identifying user preferences and changing sett...
79,10169323,2019-01-01,Diagnosing autism spectrum disorder using natu...,Embodiments herein include a natural language ...,org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Rochester', 'inventor_coun...",Diagnosing autism spectrum disorder using natu...
80,10169326,2019-01-01,Cognitive reminder notification mechanisms for...,A data processing system generates a result of...,org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Morrisville', 'inventor_co...",Cognitive reminder notification mechanisms for...
81,10169327,2019-01-01,Cognitive reminder notification mechanisms for...,A data processing system generates a result of...,org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Morrisville', 'inventor_co...",Cognitive reminder notification mechanisms for...
...,...,...,...,...,...,...,...,...,...,...,...
11,10229687,2019-03-12,Scalable endpoint-dependent natural language u...,A computer-implemented technique is described ...,org_EilEWQcC6UiqHcSGx9mb,...,2019,utility,B2,"[{'inventor_city': 'Sammamish', 'inventor_coun...",Scalable endpoint-dependent natural language u...
12,10230677,2019-03-12,Identifying an entity associated with an onlin...,An approach is described for identifying an en...,org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Sewickley', 'inventor_coun...",Identifying an entity associated with an onlin...
13,10230680,2019-03-12,Intelligently splitting text in messages poste...,"A method, system and computer program product ...",org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Shanghai', 'inventor_count...",Intelligently splitting text in messages poste...
7,10229673,2019-03-12,System and method for providing follow-up resp...,"In certain implementations, follow-up response...",org_9D8x1qL3IRASp6GG7Glu,...,2019,utility,B2,"[{'inventor_city': 'Bellevue', 'inventor_count...",System and method for providing follow-up resp...


In [42]:
text_data = df.patent_title_abstract.tolist()
text_data[:3]

['Initializing a workspace for building a natural language understanding system Designing a natural language understanding (NLU) model for an application from scratch can be difficult for non-experts. A system can simplify the design process by providing an interface allowing a designer to input example usage sentences and build an NLU model based on presented matches to those example sentences. In one embodiment, a method for initializing a workspace for building an NLU system includes parsing a sample sentence to select at least one candidate stub grammar from among multiple candidate stub grammars. The method can include presenting, to a user, respective representations of the candidate stub grammars selected by the parsing of the sample sentence. The method can include enabling the user to choose one of the respective representations of the candidate stub grammars. The method can include adding to the workspace a stub grammar corresponding to the representation of the candidate stu

In [43]:
# partition data
len(text_data)
text_train = text_data[:round(len(text_data)*.8)]
text_test = text_data[round(len(text_data)*.8):]
print(len(text_data), len(text_train), len(text_test), len(text_train)+len(text_test))

100 80 20 100


### Pre-process text data

In [44]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

#### Tokenize

In [45]:
# tokenize documents

def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

tokenized_docs = tokenize_docs(text_train)

#### Clean punctuation

In [46]:
# clean punctuation
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [47]:
cleaned_data = clean_docs(tokenized_docs)
cleaned_data[0]

['Initializing',
 'a',
 'workspace',
 'for',
 'building',
 'a',
 'natural',
 'language',
 'understanding',
 'system',
 'Designing',
 'a',
 'natural',
 'language',
 'understanding',
 'NLU',
 'model',
 'for',
 'an',
 'application',
 'from',
 'scratch',
 'can',
 'be',
 'difficult',
 'for',
 'A',
 'system',
 'can',
 'simplify',
 'the',
 'design',
 'process',
 'by',
 'providing',
 'an',
 'interface',
 'allowing',
 'a',
 'designer',
 'to',
 'input',
 'example',
 'usage',
 'sentences',
 'and',
 'build',
 'an',
 'NLU',
 'model',
 'based',
 'on',
 'presented',
 'matches',
 'to',
 'those',
 'example',
 'sentences',
 'In',
 'one',
 'embodiment',
 'a',
 'method',
 'for',
 'initializing',
 'a',
 'workspace',
 'for',
 'building',
 'an',
 'NLU',
 'system',
 'includes',
 'parsing',
 'a',
 'sample',
 'sentence',
 'to',
 'select',
 'at',
 'least',
 'one',
 'candidate',
 'stub',
 'grammar',
 'from',
 'among',
 'multiple',
 'candidate',
 'stub',
 'grammars',
 'The',
 'method',
 'can',
 'include',
 'presen

#### Convert to lowercase

In [48]:
# convert to lowercase
def lower_words(docs):
    lowered_words = []
    for doc in docs:
        lowered_words.append([word.lower() for word in doc])
    return lowered_words

lowered_data = lower_words(cleaned_data)
lowered_data[0]

['initializing',
 'a',
 'workspace',
 'for',
 'building',
 'a',
 'natural',
 'language',
 'understanding',
 'system',
 'designing',
 'a',
 'natural',
 'language',
 'understanding',
 'nlu',
 'model',
 'for',
 'an',
 'application',
 'from',
 'scratch',
 'can',
 'be',
 'difficult',
 'for',
 'a',
 'system',
 'can',
 'simplify',
 'the',
 'design',
 'process',
 'by',
 'providing',
 'an',
 'interface',
 'allowing',
 'a',
 'designer',
 'to',
 'input',
 'example',
 'usage',
 'sentences',
 'and',
 'build',
 'an',
 'nlu',
 'model',
 'based',
 'on',
 'presented',
 'matches',
 'to',
 'those',
 'example',
 'sentences',
 'in',
 'one',
 'embodiment',
 'a',
 'method',
 'for',
 'initializing',
 'a',
 'workspace',
 'for',
 'building',
 'an',
 'nlu',
 'system',
 'includes',
 'parsing',
 'a',
 'sample',
 'sentence',
 'to',
 'select',
 'at',
 'least',
 'one',
 'candidate',
 'stub',
 'grammar',
 'from',
 'among',
 'multiple',
 'candidate',
 'stub',
 'grammars',
 'the',
 'method',
 'can',
 'include',
 'presen

#### Clean stopwords

In [49]:
# clean stopwords

stop_words = stopwords.words('english')

In [50]:
def filter_stopwords(docs):
    filtered_docs = []
    for doc in docs:
       filtered_docs.append([word for word in doc if word not in stop_words])
    return filtered_docs

# remove stopwords
filtered_data = filter_stopwords(lowered_data)
filtered_data
# TODO (Lee) - resolve un-lowered stopwords "A" and "An", 'By', 'The'

[['initializing',
  'workspace',
  'building',
  'natural',
  'language',
  'understanding',
  'system',
  'designing',
  'natural',
  'language',
  'understanding',
  'nlu',
  'model',
  'application',
  'scratch',
  'difficult',
  'system',
  'simplify',
  'design',
  'process',
  'providing',
  'interface',
  'allowing',
  'designer',
  'input',
  'example',
  'usage',
  'sentences',
  'build',
  'nlu',
  'model',
  'based',
  'presented',
  'matches',
  'example',
  'sentences',
  'one',
  'embodiment',
  'method',
  'initializing',
  'workspace',
  'building',
  'nlu',
  'system',
  'includes',
  'parsing',
  'sample',
  'sentence',
  'select',
  'least',
  'one',
  'candidate',
  'stub',
  'grammar',
  'among',
  'multiple',
  'candidate',
  'stub',
  'grammars',
  'method',
  'include',
  'presenting',
  'user',
  'respective',
  'representations',
  'candidate',
  'stub',
  'grammars',
  'selected',
  'parsing',
  'sample',
  'sentence',
  'method',
  'include',
  'enabling',
 

#### Create dictionary and convert tokens into frequency counts by document

In [51]:
# specify corpus - list of patent-list of tokenized words
texts = filtered_data

In [52]:
filtered_data

[['initializing',
  'workspace',
  'building',
  'natural',
  'language',
  'understanding',
  'system',
  'designing',
  'natural',
  'language',
  'understanding',
  'nlu',
  'model',
  'application',
  'scratch',
  'difficult',
  'system',
  'simplify',
  'design',
  'process',
  'providing',
  'interface',
  'allowing',
  'designer',
  'input',
  'example',
  'usage',
  'sentences',
  'build',
  'nlu',
  'model',
  'based',
  'presented',
  'matches',
  'example',
  'sentences',
  'one',
  'embodiment',
  'method',
  'initializing',
  'workspace',
  'building',
  'nlu',
  'system',
  'includes',
  'parsing',
  'sample',
  'sentence',
  'select',
  'least',
  'one',
  'candidate',
  'stub',
  'grammar',
  'among',
  'multiple',
  'candidate',
  'stub',
  'grammars',
  'method',
  'include',
  'presenting',
  'user',
  'respective',
  'representations',
  'candidate',
  'stub',
  'grammars',
  'selected',
  'parsing',
  'sample',
  'sentence',
  'method',
  'include',
  'enabling',
 

In [57]:
# build dictionary – a mapping between words and their integer ids
id_to_word = corpora.Dictionary(filtered_data)
# id_to_word[]

In [58]:
# .dfs returns frequency of documents containing given token in tuple (token_id, count of documents that contain this token)
id_to_word.dfs.items()

dict_items([(22, 1), (54, 1), (6, 3), (31, 80), (25, 80), (51, 7), (50, 45), (13, 1), (32, 4), (29, 11), (3, 12), (43, 1), (14, 1), (48, 1), (11, 3), (37, 7), (38, 13), (24, 15), (1, 2), (12, 1), (23, 19), (17, 11), (52, 2), (47, 2), (5, 2), (4, 43), (35, 2), (27, 6), (33, 43), (15, 6), (28, 37), (21, 18), (34, 7), (42, 1), (46, 6), (44, 1), (26, 21), (7, 2), (49, 1), (18, 2), (2, 6), (30, 7), (19, 1), (20, 31), (36, 3), (53, 40), (41, 3), (40, 3), (45, 5), (16, 1), (8, 1), (0, 3), (10, 13), (39, 9), (9, 1), (77, 1), (56, 1), (82, 5), (68, 15), (79, 20), (55, 6), (65, 6), (57, 16), (72, 2), (62, 1), (78, 1), (66, 2), (71, 41), (73, 14), (63, 14), (81, 4), (61, 13), (64, 17), (74, 7), (69, 9), (76, 4), (80, 10), (75, 9), (59, 1), (60, 7), (58, 1), (70, 2), (67, 1), (99, 2), (88, 13), (90, 2), (121, 5), (98, 2), (103, 4), (86, 2), (119, 2), (123, 2), (97, 3), (102, 2), (120, 3), (101, 9), (95, 6), (122, 2), (87, 2), (91, 7), (83, 19), (96, 2), (109, 2), (117, 2), (105, 4), (112, 3), (116

In [59]:
# apply term document frequency - convert docs in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus_train = [id_to_word.doc2bow(text) for text in texts]
len(corpus_train)

80

In [60]:
# inspect formatted term-doc frequency for word in text in corpus
[[(id_to_word[id], freq) for id, freq in text] for text in corpus_train]

[[('adding', 1),
  ('allowing', 1),
  ('among', 1),
  ('application', 1),
  ('based', 1),
  ('build', 1),
  ('building', 2),
  ('candidate', 5),
  ('choose', 1),
  ('chosen', 1),
  ('corresponding', 1),
  ('design', 1),
  ('designer', 1),
  ('designing', 1),
  ('difficult', 1),
  ('embodiment', 1),
  ('enabling', 1),
  ('example', 2),
  ('grammar', 3),
  ('grammars', 3),
  ('include', 3),
  ('includes', 1),
  ('initializing', 2),
  ('input', 1),
  ('interface', 1),
  ('language', 2),
  ('least', 1),
  ('matches', 1),
  ('method', 4),
  ('model', 2),
  ('multiple', 1),
  ('natural', 2),
  ('nlu', 3),
  ('one', 3),
  ('parsing', 2),
  ('presented', 1),
  ('presenting', 1),
  ('process', 1),
  ('providing', 1),
  ('representation', 1),
  ('representations', 2),
  ('respective', 2),
  ('sample', 2),
  ('scratch', 1),
  ('select', 1),
  ('selected', 1),
  ('sentence', 2),
  ('sentences', 2),
  ('simplify', 1),
  ('stub', 6),
  ('system', 3),
  ('understanding', 2),
  ('usage', 1),
  ('user'

### Model - model #1

In [114]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus_train,
                     id2word=id_to_word,
                     num_topics=10, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

In [115]:
# # extracts topics for given document from Gensim
# def get_topics(doc, k=5, model_lda=model_lda):
#     topic_id = sorted(model_lda[doc][0], key=lambda x: -x[1])
#     top_k_topics = [x[0] for x in topic_id[:k]]
#     return [(i, model_lda.print_topic(i)) for i in top_k_topics]

In [116]:
# `get_document_topics()` returns topic probability distribution for given document
topic_dist_675_a = model_lda.get_document_topics(corpus_train[15])
pprint(sorted(topic_dist_675_a))

[(1, 0.9003208), (8, 0.09514114)]


In [117]:
topicid = 3
model_lda.get_topic_terms(topicid, topn=10)

[(194, 0.025315898),
 (25, 0.024299867),
 (53, 0.024268024),
 (31, 0.023171464),
 (561, 0.023042997),
 (26, 0.016707495),
 (33, 0.016240673),
 (164, 0.016151952),
 (163, 0.014989183),
 (327, 0.013792267)]

In [118]:
text_train[doc_id]

'Knowledge-based editor with natural language interface A computer-implemented method for knowledge based ontology editing, is provided. The method receives a language instance to update a knowledge base, using a computer. The method semantically parses the language instance to detect an ontology for editing. The method maps one or more nodes for the ontology for editing based on an ontology database and the knowledge base. The method determines whether the mapped nodes are defined or undefined within the knowledge base. The method calculates a first confidence score based on a number of the defined and undefined mapped nodes. Furthermore, the method updates the knowledge base when the first confidence score meets a pre-defined threshold.'

In [119]:
doc_id = 15
topic_dist_15_b = sorted(get_topics(corpus_train[doc_id], k=10)), text_train[doc_id]
pprint(topic_dist_15_b)

TypeError: get_topics() missing 1 required positional argument: 'model_lda'

In [107]:
# text = 'virtual dictionary lexicon enablement voice'.split()
text_input = 'smart assistant transformer model translation'.split()

In [78]:
id_to_word.doc2bow(text_input)

[(29, 1), (121, 1), (197, 1)]

In [101]:
get_topics(id_to_word.doc2bow(text_input), k=10)

TypeError: get_topics() missing 1 required positional argument: 'model_lda'

In [80]:
def get_documents()

SyntaxError: invalid syntax (<ipython-input-80-3f100f9287da>, line 1)

In [81]:
text_train[0]

'Initializing a workspace for building a natural language understanding system Designing a natural language understanding (NLU) model for an application from scratch can be difficult for non-experts. A system can simplify the design process by providing an interface allowing a designer to input example usage sentences and build an NLU model based on presented matches to those example sentences. In one embodiment, a method for initializing a workspace for building an NLU system includes parsing a sample sentence to select at least one candidate stub grammar from among multiple candidate stub grammars. The method can include presenting, to a user, respective representations of the candidate stub grammars selected by the parsing of the sample sentence. The method can include enabling the user to choose one of the respective representations of the candidate stub grammars. The method can include adding to the workspace a stub grammar corresponding to the representation of the candidate stub

In [82]:
model_lda[doc]

NameError: name 'doc' is not defined

In [83]:
model_lda.do_estep(chunk, state=None)

NameError: name 'chunk' is not defined

In [84]:
# print keywords in n topics
sorted(model_lda.show_topics(), key=lambda x: x[1])

[(5,
  '0.020*"segments" + 0.020*"messages" + 0.020*"message" + 0.020*"subsystem" + 0.020*"item" + 0.020*"linguistic" + 0.017*"user" + 0.017*"based" + 0.017*"social" + 0.017*"manner"'),
 (3,
  '0.025*"may" + 0.024*"language" + 0.024*"user" + 0.023*"natural" + 0.023*"domain" + 0.017*"least" + 0.016*"one" + 0.016*"query" + 0.015*"information" + 0.014*"plurality"'),
 (9,
  '0.026*"information" + 0.019*"system" + 0.017*"may" + 0.016*"method" + 0.016*"one" + 0.015*"query" + 0.014*"generating" + 0.014*"natural" + 0.014*"language" + 0.013*"based"'),
 (6,
  '0.026*"language" + 0.025*"method" + 0.024*"natural" + 0.024*"include" + 0.022*"may" + 0.022*"entity" + 0.016*"one" + 0.015*"processing" + 0.015*"system" + 0.014*"communication"'),
 (2,
  '0.032*"language" + 0.028*"natural" + 0.019*"application" + 0.017*"query" + 0.016*"one" + 0.016*"cad" + 0.016*"method" + 0.012*"sound" + 0.012*"database" + 0.012*"user"'),
 (1,
  '0.035*"method" + 0.027*"knowledge" + 0.022*"ontology" + 0.022*"base" + 0.016

In [85]:
# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[1])

[(5,
  '0.020*"segments" + 0.020*"messages" + 0.020*"message" + 0.020*"subsystem" + 0.020*"item" + 0.020*"linguistic" + 0.017*"user" + 0.017*"based" + 0.017*"social" + 0.017*"manner"'),
 (3,
  '0.025*"may" + 0.024*"language" + 0.024*"user" + 0.023*"natural" + 0.023*"domain" + 0.017*"least" + 0.016*"one" + 0.016*"query" + 0.015*"information" + 0.014*"plurality"'),
 (9,
  '0.026*"information" + 0.019*"system" + 0.017*"may" + 0.016*"method" + 0.016*"one" + 0.015*"query" + 0.014*"generating" + 0.014*"natural" + 0.014*"language" + 0.013*"based"'),
 (6,
  '0.026*"language" + 0.025*"method" + 0.024*"natural" + 0.024*"include" + 0.022*"may" + 0.022*"entity" + 0.016*"one" + 0.015*"processing" + 0.015*"system" + 0.014*"communication"'),
 (2,
  '0.032*"language" + 0.028*"natural" + 0.019*"application" + 0.017*"query" + 0.016*"one" + 0.016*"cad" + 0.016*"method" + 0.012*"sound" + 0.012*"database" + 0.012*"user"'),
 (1,
  '0.035*"method" + 0.027*"knowledge" + 0.022*"ontology" + 0.022*"base" + 0.016

In [86]:
# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[1])

[(5,
  '0.020*"segments" + 0.020*"messages" + 0.020*"message" + 0.020*"subsystem" + 0.020*"item" + 0.020*"linguistic" + 0.017*"user" + 0.017*"based" + 0.017*"social" + 0.017*"manner"'),
 (3,
  '0.025*"may" + 0.024*"language" + 0.024*"user" + 0.023*"natural" + 0.023*"domain" + 0.017*"least" + 0.016*"one" + 0.016*"query" + 0.015*"information" + 0.014*"plurality"'),
 (9,
  '0.026*"information" + 0.019*"system" + 0.017*"may" + 0.016*"method" + 0.016*"one" + 0.015*"query" + 0.014*"generating" + 0.014*"natural" + 0.014*"language" + 0.013*"based"'),
 (6,
  '0.026*"language" + 0.025*"method" + 0.024*"natural" + 0.024*"include" + 0.022*"may" + 0.022*"entity" + 0.016*"one" + 0.015*"processing" + 0.015*"system" + 0.014*"communication"'),
 (2,
  '0.032*"language" + 0.028*"natural" + 0.019*"application" + 0.017*"query" + 0.016*"one" + 0.016*"cad" + 0.016*"method" + 0.012*"sound" + 0.012*"database" + 0.012*"user"'),
 (1,
  '0.035*"method" + 0.027*"knowledge" + 0.022*"ontology" + 0.022*"base" + 0.016

In [87]:
# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[0])

[(0,
  '0.042*"system" + 0.025*"response" + 0.023*"may" + 0.021*"natural" + 0.021*"language" + 0.020*"one" + 0.016*"method" + 0.014*"include" + 0.014*"using" + 0.013*"user"'),
 (1,
  '0.035*"method" + 0.027*"knowledge" + 0.022*"ontology" + 0.022*"base" + 0.016*"nodes" + 0.016*"editing" + 0.015*"based" + 0.011*"language" + 0.011*"undefined" + 0.011*"defined"'),
 (2,
  '0.032*"language" + 0.028*"natural" + 0.019*"application" + 0.017*"query" + 0.016*"one" + 0.016*"cad" + 0.016*"method" + 0.012*"sound" + 0.012*"database" + 0.012*"user"'),
 (3,
  '0.025*"may" + 0.024*"language" + 0.024*"user" + 0.023*"natural" + 0.023*"domain" + 0.017*"least" + 0.016*"one" + 0.016*"query" + 0.015*"information" + 0.014*"plurality"'),
 (4,
  '0.036*"may" + 0.027*"language" + 0.024*"natural" + 0.023*"user" + 0.021*"data" + 0.017*"based" + 0.015*"set" + 0.013*"query" + 0.010*"system" + 0.010*"input"'),
 (5,
  '0.020*"segments" + 0.020*"messages" + 0.020*"message" + 0.020*"subsystem" + 0.020*"item" + 0.020*"lin

In [88]:
# show_topic() returns n most important/relevant words, and their weights, that comprise given topic
pprint(model_lda.show_topic(1, topn=10))

[('method', 0.034632463),
 ('knowledge', 0.026877712),
 ('ontology', 0.021603875),
 ('base', 0.021572566),
 ('nodes', 0.016070455),
 ('editing', 0.016052011),
 ('based', 0.015080399),
 ('language', 0.011330525),
 ('undefined', 0.011033404),
 ('defined', 0.011001011)]


In [89]:
pprint(model_lda.show_topics(num_topics=5, num_words=10))

[(1,
  '0.035*"method" + 0.027*"knowledge" + 0.022*"ontology" + 0.022*"base" + '
  '0.016*"nodes" + 0.016*"editing" + 0.015*"based" + 0.011*"language" + '
  '0.011*"undefined" + 0.011*"defined"'),
 (5,
  '0.020*"segments" + 0.020*"messages" + 0.020*"message" + 0.020*"subsystem" + '
  '0.020*"item" + 0.020*"linguistic" + 0.017*"user" + 0.017*"based" + '
  '0.017*"social" + 0.017*"manner"'),
 (0,
  '0.042*"system" + 0.025*"response" + 0.023*"may" + 0.021*"natural" + '
  '0.021*"language" + 0.020*"one" + 0.016*"method" + 0.014*"include" + '
  '0.014*"using" + 0.013*"user"'),
 (8,
  '0.043*"user" + 0.042*"natural" + 0.042*"language" + 0.030*"one" + '
  '0.020*"input" + 0.014*"may" + 0.014*"processing" + 0.014*"device" + '
  '0.013*"system" + 0.013*"computer"'),
 (4,
  '0.036*"may" + 0.027*"language" + 0.024*"natural" + 0.023*"user" + '
  '0.021*"data" + 0.017*"based" + 0.015*"set" + 0.013*"query" + 0.010*"system" '
  '+ 0.010*"input"')]


### Evaluate - model #1

In [91]:
# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus_train)
perplexity

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

-6.2473350707784165

In [92]:
# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=filtered_data, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

0.35195810601775207

In [94]:
# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

[0.26568837912073745,
 0.8694071050796905,
 0.28677165120015713,
 0.26416435302382346,
 0.29694091734843053,
 0.3771963570118807,
 0.3062411668133168,
 0.27085407729720307,
 0.24953739438762645,
 0.33277965889465444]

In [97]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus_train, id_to_word)
viz_topics_1
# TODO (Lee) - salient vs relevant terms in pyLDA ?

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.045530  0.066325       1        1  15.517809
0     -0.068495  0.011624       2        1  14.155208
8     -0.050546  0.038696       3        1  13.505877
6     -0.025301 -0.221956       4        1  13.350457
3     -0.063279  0.047213       5        1  12.018886
7     -0.064273  0.018142       6        1  11.199162
9     -0.012979  0.010014       7        1   9.223829
2     -0.010018  0.004150       8        1   7.197275
5      0.201050  0.014377       9        1   2.797030
1      0.139370  0.011416      10        1   1.034468, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
28    Default   80.000000         method   80.000000  30.0000  30.0000
88    Default   58.000000        content   58.000000  29.0000  29.0000
136   Default   16.000000      knowledge   16.000000  28.0000  28.0000
53    Default  104.000000           user  104.000000  27.0000  27.0000
561   Default   18.000000         domain   18.000000  26.0000  26.0000
...       ...         ...            ...         ...      ...      ...
358   Topic10    0.034159    relatedness    1.355383   0.8905  -7.5374
346   Topic10    0.034155  intelligently    1.355262   0.8905  -7.5375
4     Topic10    0.966900          based   63.272831   0.3902  -4.1944
24    Topic10    0.355414      interface   23.356455   0.3859  -5.1952
25    Topic10    0.726472       language  176.823242  -0.9234  -4.4803

[601 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1130      4  0.799817  accountability
469       5  0.805423           added
188       3  0.888083        adequate
965       1  0.667991           alert
965       5  0.333996           alert
...     ...       ...             ...
196       5  0.174463         whether
196       7  0.058154         whether
1118      5  0.805384        wireless
586       2  0.934987        workflow
54        3  0.888067       workspace

[978 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 9, 7, 4, 8, 10, 3, 6, 2])

### Model 2-  Mallet model

In [120]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update this path
path_mallet = 'data/mallet-2.0.8/bin/mallet'

In [121]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus_train, num_topics=25, id2word=id_to_word)

CalledProcessError: Command 'data/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/6g/27zvvxsd0v12h5nfky3vldz00000gn/T/3e20a2_corpus.txt --output /var/folders/6g/27zvvxsd0v12h5nfky3vldz00000gn/T/3e20a2_corpus.mallet' returned non-zero exit status 127.

In [123]:
# topics
pprint(model_2.show_topics(formatted=False))

NameError: name 'model_2' is not defined

In [124]:
# calculate coherence metric
coherence_model_2 = CoherenceModel(model=model_2, texts=filtered_data, dictionary=id_to_word, coherence='c_v')
coherence_model_2 = coherence_model_2.get_coherence()
coherence_model_2

NameError: name 'model_2' is not defined

### Model 3 - Author topic model

#### pre-process

In [125]:
tokenized_docs_at = tokenize_docs(text_data)
cleaned_data_at = clean_docs(tokenized_docs_at)
lowered_data_at = lower_words(cleaned_data_at)
filtered_data_at = filter_stopwords(lowered_data_at)

In [126]:
len(filtered_data_at)

100

#### build dictionary and corpus from processed text

In [127]:
# build dictionary
id_to_word_at = corpora.Dictionary(filtered_data_at)

# build corpus
texts_at = filtered_data_at

# apply term document frequency - converts docs in corpus to bag-of-words format via list of (token_id, token_count) tuples
corpus_at = [id_to_word_at.doc2bow(text) for text in texts_at]

In [128]:
(next(iter(id_to_word_at.items())))

(0, 'adding')

In [129]:
type(id_to_word_at.keys()[0])

int

In [130]:
type(id_to_word_at.values())

collections.abc.ValuesView

#### construct inventor-doc mapping from nested inventors column in json api response

In [131]:
# extract nested inventors table from api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.pop("patent_date")
df_inventors.head(3)

,inventor_id,patent_number
0,7788103-1,10229106
1,8352247-1,10229109
2,8515750-2,10229109


In [132]:
# TODO (Lee) - resolve workaround
# df_idx = df
# df_idx['idx'] = df.index
# df_idx
# # df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
# df_idx_2 = df_idx_1.set_index('patent_number')
# df_idx_2.pop('inventors')
# df_idx_2
# df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

# df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

In [133]:
dict_pat2inv =df_inventors.set_index('patent_number').T.to_dict('list')
# dict_pat2inv

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [134]:
# for k, v in pat2inv.items():
#     name_dict[new_key] = name_dict.pop(k)
#     time.sleep(4)

# pprint.pprint(name_dict)

# d = {'x':1, 'y':2, 'z':3}
# d1 = {'x':'a', 'y':'b', 'z':'c'}

# dict((d1[key], value) for (key, value) in d.items())
# {'a': 1, 'b': 2, 'c': 3}

# idx_pat_map = df.patent_number.to_dict()
# idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
# import itertools
# x = list(itertools.islice(idx_pat_map.items(), 0, 4))
# x[:4]

In [135]:
pat2inv_dict = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}

In [136]:
# {k: list(v) for k,v in df_pat2inv.groupby("patent_number")["inventor_id"]}

In [137]:
# df2 = df_inventors.groupby("patent_number")["inventor_id"]

In [138]:
# df3 = df_idx_pat_inv_map.groupby("patent_number")["inventor_id"]

In [139]:
pat2inv = {k: list(str(v)) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
len(pat2inv.items())

100

In [140]:
pat2inv.items()
type(next(iter(pat2inv)))

str

In [141]:
pat2inv_2 = {str(k): list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
len(pat2inv_2)

100

In [142]:
patdf2inv_2 = dict((df_inventors[key], value) for (key, value) in pat2inv.items())
patdf2inv_2

KeyError: '10162315'

In [143]:
patdf2inv = dict((df_pat_idx[key], value) for (key, value) in pat2inv.items())
patdf2inv

NameError: name 'df_pat_idx' is not defined

#### Construct author-topic model

In [144]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus_at,
                         doc2author=patdf2inv,
                         id2word=id_to_word_at, 
                         num_topics=25)

NameError: name 'patdf2inv' is not defined

In [145]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

NameError: name 'model_at' is not defined

In [146]:
# inspect topic distribution for author with id# 7788103-1
# each topic has a probability of being expressed given the particular inventor, but only the ones above a certain threshold are shown.

model_at['7788103-1']

NameError: name 'model_at' is not defined

In [147]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [148]:
# calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

build doc2author dictionary

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

SyntaxError: invalid syntax (<ipython-input-148-dd2860f6033b>, line 3)

In [149]:

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

NameError: name 'model' is not defined

In [150]:
gensim.models.atmodel.construct_author2doc(doc2author)
# construct mapping from author IDs to document IDs

Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
Returns:	Mapping of authors to document ids.
Return type:	dict of (str, list of int)

SyntaxError: invalid syntax (<ipython-input-150-51495fa4aa4b>, line 4)

In [151]:
gensim.models.atmodel.construct_doc2author(corpus, author2doc)
construct mapping from document IDs to author IDs

Parameters:	
corpus (iterable of list of (int, float)) – Corpus in BoW format.
author2doc (dict of (str, list of int)) – Mapping of authors to documents.
Returns:	
Document to Author mapping.

Return type:	
dict of (int, list of str)

SyntaxError: invalid syntax (<ipython-input-151-ef53f61374fe>, line 2)

### Appendix

#### Import full dataset from PatentsView API

In [ ]:
# uncomment to use

# def get_patents_by_month(begin_date,end_date, pats_per_page):
#     """ requests patent data from PatentsView API by date range"""
#     endpoint_url = 'http://www.patentsview.org/api/patents/query'
#     page_counter=1
#     data = []
#     results = {}
#     count=1
    
#     for i in range(round(100000/pats_per_page)): # TODO (Lee) - replace with datetime for begin_date to end_date
        
#         if count ==0:
#             print("error/complete")
#             break
            
#         elif count > 0:     
#             # build query
#             query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
#             fields=pat_fields
#             options={"page": page_counter, "per_page":pats_per_page}
#             sort=[{"patent_date":"desc"}]
#             params={'q': json.dumps(query),
#                     'f': json.dumps(fields),
#                     'o': json.dumps(options),
#                     's': json.dumps(sort)
#                         }
    
#             # request and results
#             response = requests.get(endpoint_url, params=params)
#             status = response.status_code
#             print("status:", status,';',"page_counter:",page_counter, ";", "iteration:",i)
#             results = response.json()
#             count = results.get("count")
#             total_pats = results.get("total_patent_count")
#             print("patents on current page:",count,';', "total patents:",total_pats)
#             data.extend(results)
#             page_counter+=1
        
#     return data
#             # TODO (Lee) results =  json.loads(response.content)
#             # TODO (Lee) places.extend(results['results'])
#             # TODO (Lee) time.sleep(2)

CPC fields for block 1 of query:
Y10S-706 OR 
G06N-003 OR 
G06N-005/003:G06N-005/027 OR 
G06N- 007/005:G06N-007/06 OR 
G06N-099/005 OR
G06T2207/20081 OR
G06T2207/20084 OR
G06T-003/4046 OR
G06T-009/002 OR
G06F-017/16 OR
G05B-013/027 OR
G05B- 013/0275 OR
G05B-013/028 OR
G05B-013/0285 OR
G05B-013/029 OR
G05B-013/0295 OR
G05B-2219/33002 OR
G05D-001/0088 OR
G06K-009 OR
G10L-015 OR
G10L-017 OR
G06F-017/27:G06F-017/2795 OR
G06F-017/28:G06F-017/289 OR
G06F-017/30029:G06F- 017/30035 OR
G06F-017/30247:G06F-017/30262 OR 
G06F-017/30401 OR
G06F-017/3043 OR 
G06F-017/30522:G06F-017/3053 OR 
G06F-017/30654 OR 
G06F-017/30663 OR
G06F-017/30666 OR 
G06F-017/30669 OR
G06F-017/30672 OR 
G06F-017/30684 OR
G06F-017/30687 OR 
G06F-017/3069 OR 
G06F-017/30702 OR
G06F-017/30705:G06F- 017/30713 OR
G06F-017/30731:G06F-017/30737 OR
G06F-017/30743:G06F-017/30746 OR 
G06F-017/30784:G06F-017/30814 OR
G06F-019/24 OR G06F-019/707 OR
G01R- 031/2846:G01R-031/2848 OR
G01N-2201/1296 OR
G01N-029/4481 OR
G01N-033/0034 ORG01R-031/3651ORG01S-007/417ORG06N-003/004:G06N-003/008 ORG06F- 011/1476 OR 
G06F-011/2257 OR 
G06F-011/2263 OR 
G06F-015/18 OR
G06F-2207/4824 OR
G06K-007/1482 OR
G06N-007/046 OR
G11B-020/10518 OR
G10H-2250/151 OR
G10H-2250/311 OR
G10K-2210/3024 OR
H01J-2237/30427 OR
H01M-008/04992 OR
H02H-001/0092 OR
H02P-021/0014 OR
H02P-023/0018 OR
H03H-2017/0208 OR
H03H- 2222/04 OR
H04L-2012/5686 OR
H04L-2025/03464 OR
H04L-2025/03554 OR
H04L- 025/0254 OR
H04L-025/03165 OR
H04L-041/16 OR
H04L-045/08 OR
H04N- 021/4662:H04N-021/4666 OR
H04Q-2213/054 
OR H04Q-2213/13343 OR
H04Q-2213/343 OR
H04R-025/507 OR
G08B-029/186 OR
B60G-2600/1876 OR
B60G-2600/1878 OR
B60G-2600/1879 OR
B64G-2001/247 OR
E21B-2041/0028 OR
B23K-031/006 OR
B29C- 2945/76979 OR
B29C-066/965 OR
B25J-009/161 OR
A61B-005/7264:A61B-005/7267 OR
Y10S-128/924 OR
Y10S-128/925 OR
F02D-041/1405 OR
F03D-007/046 OR
F05B- 2270/707 OR
F05B-2270/709 OR
F16H-2061/0081 OR
F16H-2061/0084 OR
B60W-030/06 OR
B60W-030/10:B60W-030/12 OR
B60W-030/14:B60W-030/17 OR
B62D-015/0285 OR
G06T-2207/30248:G06T-2207/30268 OR
G06T-2207/30236 OR G05D-001 OR
A61B- 005/7267 OR
F05D-2270/709 OR
G06T-2207/20084 OR
G10K-2210/3038 OR
G10L-025/30 OR
H04N-021/4666 OR
A63F-013/67 OR
G06F-017/2282

#### Import data from bulk download

In [ ]:
# uncomment to download TSV files containing detailed patent descriptions from PatentsView 
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2016.tsv.zip # 2016 - 3.0 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2017.tsv.zip # 2017 - 2.8 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2018.tsv.zip # 2018 - 1.6 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2019.tsv.zip # 2019 - 0.7 GB zipped

# !unzip files
# unzip detail-desc-text-2016.tsv.zip
# unzip detail-desc-text-2017.tsv.zip
# unzip detail-desc-text-2018.tsv.zip
# unzip detail-desc-text-2019.tsv.zip

# def convert_bytes(num, suffix='B'):
#     """ convert bytes int to int in aggregate units"""
#     for unit in ['','K','M','G','T','P','E','Z']:
#         if abs(num) < 1024.0:
#             return "%3.1f%s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f%s%s" % (num, 'Yi', suffix)

# path = "data/"
# with os.scandir(path) as it:
#     for entry in it:
#         if not entry.name.startswith('.') and entry.is_file():
#             print(entry.name)

# # inspect unzipped file sizes

# convert_bytes(os.path.getsize("data/detail-desc-text-2016.tsv"))
# convert_bytes(os.path.getsize("data/detail-desc-text-2017.tsv"))
# convert_bytes(os.path.getsize("data/detail-desc-text-2018.tsv"))
# convert_bytes(os.path.getsize("data/detail_desc_text_2019.tsv"))

#### Spark workflow

# create SparkSession/SparkContext as entry point to Dataset/DataFrame API

# spark = pyspark.sql.SparkSession.builder.getOrCreate()
# sc = spark.sparkContext
# sc

# doc by doc read of large tsv files

# with open "data/detail-desc-text-2018.tsv" as f_in:
#     with open('data/2018_shard.tsv', 'w') as f_out:
#         for line in f_in:
#             if contains_keywords(line):
#                 f_out.write(line)

# from functools import reduce

# df = reduce(lambda x,y: x.unionAll(y), 
#             [spark.read.format('csv')
#                        .load(f, header="true", inferSchema="true") 
#              for f in files])
# df.show()

# files = ["data/detail-desc-text-2016.tsv", "data/detail-desc-text-2017.tsv", 
#          "data/detail-desc-text-2018.tsv", "data/detail-desc-text-2019.tsv"]

# uncomment to use

# df_2018 = (spark.read
#                .format("csv")
#                .option("delimiter", "\t")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2018.tsv")
#                .write
#                .format("parquet")
#                .save("df_2018.parquet"))

# df_2018 = (spark.read
#                .format("csv")
#                .option("delimiter", "\t")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2018.tsv"))

# dfp_2018 = pd.read_csv("data/detail-desc-text-2018.tsv", sep='\t', header=None)

# dfp_2018.columns = ['patent_number', 'desc_detail', 'len_detail']

# dfp_2018_nl.head(3)

# dfp_2018_nl = dfp_2018[dfp_2018['desc_detail'].str.contains('NLP')]

# df_2018.printSchema()

# schema = StructType([
#             StructField("_c0", IntegerType(), True),
#             StructField("_c1", StringType(), True),
#             StructField("_c2", IntegerType(), True)])

# df_2018 = spark.read.load('data/df_2018.parquet')

# # 160,249 rows in 2018 dataset
# df_2018.count()

# df_2018.rdd.getNumPartitions()

# # partition / batching ?
# df_2018.filter(df_2018._c1.contains("natural language")).count()

# query file directly with SQL

# query = """
# SELECT * FROM parquet.`data/df_2018.parquet` WHERE _c1 LIKE 'natural language' LIMIT 100
# """

# df_2018_nl = spark.sql(query)

# df_2018_nl.head(3)

# df_2018.columns

# df_2018.explain()

# df_2018.describe().show()

# df_2018.dtypes

# df_171819 = df_2017.union(df_2018).union(df_2019)

# df_171819.count()

# df_171819.head(3)

# df_2018.head(3)

# counts = df_2018.agg(F.countDistinct('_c0'))
# counts

# reviews_df.createOrReplaceTempView('reviews')

# output = spark.sql(query)

# show(output, n=1000)

# results = spark.sql(
#   "SELECT * FROM people")
# names = results.map(lambda p: p.name)

# df.rdd.isEmpty()

# df = (spark.read
#             .load("data/*.parquet")
#             .write
#             .format("parquet")
#             .save("df.parquet"))

# df_2019 = (spark.read
#                .format("csv")
#                .option("delimiter", "\t")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2018.tsv")
#                .write
#                .format("parquet")
#                .save("df_2019.parquet"))

# df_2019.head(3)

# type(df_2018)

# df_2016.head(3)

# df_2016 = (spark.read.format("csv")
#                .option("delimiter", "\t")
#                .option("header", "true")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2016.tsv")
#                .write
#                .format("parquet")
#                .save("data/df_2016.parquet"))

# df_2016.head(3)

# df = (spark.read.format("csv")
#            .option("delimiter", ",")
#            .infer
#            .load("data/df.csv"))

# df_2017 = (spark.read.format("csv")
#                .option("delimiter", "\t")
#                .option("header", "true")
#                .option('inferSchema', "true")
#                .load("data/detail-desc-text-2017.tsv")
#                .write
#                .format("parquet")
#                .save("df_2017.parquet"))

# df_2018 = spark.read.parquet("data/df_2018.parquet")

# df_2018.head(2)

# df_2018.persist()

# df_2018.take(2)

# df_2018.toPandas()

#### Construct bigrams and trigrams

In [62]:
# train bigram phrases model
bigram_model = Phrases(filtered_data, min_count=1, threshold=1)

# train trigram phrases model
trigram_model = Phrases(bigram_model[filtered_data], threshold=100)  

# bigrams
def bigrams(docs):
    """create bigrams"""
    return [bigram_model[doc] for doc in docs]

# initialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram_model)
trigram_model = gensim.models.phrases.Phraser(trigram_model)

bigrams(filtered_data)[0]

def trigrams(docs):
    """create trigrams"""
    return [trigram_model[bigram_model[doc]] for doc in docs]

trigrams(filtered_data)[0]

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['initializing_workspace',
 'building_natural',
 'language_understanding',
 'system',
 'designing',
 'natural_language',
 'understanding_nlu',
 'model',
 'application',
 'scratch',
 'difficult',
 'system',
 'simplify',
 'design',
 'process',
 'providing',
 'interface',
 'allowing',
 'designer',
 'input',
 'example',
 'usage',
 'sentences',
 'build',
 'nlu_model',
 'based',
 'presented',
 'matches',
 'example',
 'sentences',
 'one_embodiment',
 'method',
 'initializing_workspace',
 'building',
 'nlu',
 'system_includes',
 'parsing_sample',
 'sentence',
 'select',
 'least_one',
 'candidate_stub',
 'grammar',
 'among_multiple',
 'candidate_stub',
 'grammars_method',
 'include_presenting',
 'user',
 'respective_representations',
 'candidate_stub',
 'grammars',
 'selected',
 'parsing_sample',
 'sentence_method',
 'include',
 'enabling',
 'user',
 'choose',
 'one',
 'respective_representations',
 'candidate_stub',
 'grammars_method',
 'include_adding',
 'workspace',
 'stub_grammar',
 'corres

#### Stem and Lemmatize

In [63]:
def lemmatize_docs(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """lemmatize documents"""
    lemmatized_docs = []
    for doc in docs: 
        lemmatized_docs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return lemmatized_docs

#### Alternate worklow to Create corpus and dictionary

In [ ]:
# using spacy pipeline components
# build dictionary
id_to_word = corpora.Dictionary(processed_docs)

# build corpus
texts = processed_docs

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(doc) for doc in processed_docs]

 # build dictionary
id_to_word = corpora.Dictionary(filtered_data)

# build corpus
texts = filtered_data

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(text) for text in texts]

# view formatted corpus (term-doc-frequency)
[[(id_to_word[id], freq) for id, freq in text] for text in corpus][:1]

#### Alternate workflow to pre-process text data

In [ ]:
# # uncomment to download stop words from nltk and language package from spacy
# # nltk.download('stopwords')
# # nltk.download('punkt')
# # !python -m spacy download en

# # construct pipeline using Spacy Language object and associated pipeline/components
# nlp = spacy.load("en")
# pprint(nlp.pipeline)

# processed_docs = []   

# # process patent documents in pipeline
# for doc in nlp.pipe(text_train, n_threads=4, batch_size=100):
   
#     ents = doc.ents  # Named entities.

#     # Keep only words (no numbers, no punctuation).
#     # Lemmatize tokens, remove punctuation and remove stopwords.
#     doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

#     # Remove common words from a stopword list.
#     doc = [token for token in doc if token not in stop_words]

#     # Add named entities, but only if they are a compound of more than word.
#     doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
#     processed_docs.append(doc)

# processed_docs[0][:5]

# [token.text for token in doc]

# labels = set([w.label_ for w in doc.ents]) 

# for label in labels: 
#     entities = [cleanup(e.string, lower=False) for e in document.ents if label==e.label_] 
#     entities = list(set(entities)) 
#     print(label,entities)

# pre_processed_docs = []
# for doc in nlp.pipe(docs, n_threads=4, batch_size=100):
#     # Process document using Spacy NLP pipeline.
    
#     ents = doc.ents  # Named entities.

#     # Keep only words (no numbers, no punctuation).
#     # Lemmatize tokens, remove punctuation and remove stopwords.
#     doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

#     # Remove common words from a stopword list.
#     #doc = [token for token in doc if token not in STOPWORDS]

#     # Add named entities, but only if they are a compound of more than word.
#     doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
#     pre_processed_docs.append(doc)

#### Alternate workflow to create corpus and dictionary

In [65]:
# #uncomment to use

# # using spacy pipeline components
# # build dictionary
# id_to_word = corpora.Dictionary(processed_docs)

# # build corpus
# texts = processed_docs

# # apply term document frequency
# # converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
# corpus = [id_to_word.doc2bow(doc) for doc in processed_docs]

#  # build dictionary
# id_to_word = corpora.Dictionary(filtered_data)

# # build corpus
# texts = filtered_data

# # apply term document frequency
# # converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
# corpus = [id_to_word.doc2bow(text) for text in texts]

# # view formatted corpus (term-doc-frequency)
# [[(id_to_word[id], freq) for id, freq in text] for text in corpus][:1]

#### Alternate worfklow for Model - model #1

In [ ]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_to_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

# print keywords in n topics
pprint(model_lda.print_topics())

# print top 10 keywords that comprise topic with index of 0
pprint(model_lda.print_topic(24))
# the most import keywords, and the respective weight, that form topic 0 are

# print top 10 keywords that comprise topic with index of 1
pprint(model_lda.print_topic(1))

# TODO (Lee) - infer topic from keywords?

#### Alternate workflow to Evaluate - model #1

In [ ]:
#uncomment to use

# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
perplexity

# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=processed_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=filtered_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

#### Worfklow re: coherence metric

In [ ]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_to_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

#### Alternate workflow for Model 3 - Author topic model

In [ ]:
# #uncomment to use

# # construct inventor-to-doc mapping as df from nested inventors column in json api response
# df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
# df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
# df_inventors.sort_values(by=['patent_date'])
# df_inventors.head(3)

# df.head(3)

# # TODO (Lee) - resolve workaround
# df_idx = df
# df_idx['idx'] = df.index
# df_idx
# df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
# df_idx_2 = df_idx_1.set_index('patent_number')
# df_idx_2.pop('inventors')
# df_idx_2
# df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

# df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

# df_inv_test = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
# df_inv_test.head(3)

# df_idx_pat_inv_map = df[['patent_number', 'inventors']]
# df_idx_pat_inv_map.head(3)

# # TODO (Lee) - find out how to get list of patents_view_field names from API - I did it accidentally but need to replicate

# df.patent_title_abstract[0]

# df[:3]

# df_inventors.set_index('inventor_id').T.to_dict('list')

# # for k, v in pat2inv.items():
# #     name_dict[new_key] = name_dict.pop(k)
# #     time.sleep(4)

# # pprint.pprint(name_dict)

# # d = {'x':1, 'y':2, 'z':3}
# # d1 = {'x':'a', 'y':'b', 'z':'c'}

# # dict((d1[key], value) for (key, value) in d.items())
# # {'a': 1, 'b': 2, 'c': 3}

# patdf2inv = dict((df_pat_idx[key], value) for (key, value) in pat2inv.items())
# patdf2inv

# pat2inv = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
# pat2inv

# idx_pat_map = df.patent_number.to_dict()
# idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
# idx_pat_map

# #### Construct author-topic model

# # construct author-topic model
# model_at = AuthorTopicModel(corpus=corpus,
#                          doc2author=patdf2inv,
#                          id2word=id_to_word, 
#                          num_topics=25)

# # construct vectors for authors
# author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
# author_vecs

# # retrieve the topic distribution for an author using use model[name] syntax
# # each topic has a probability of being expressed given the particular author, but only the ones above a certain threshold are shown.

# model_at['7788103-1']

# # def show_author(name):
# #     print('\n%s' % name)
# #     print('Docs:', model.author2doc[name])
# #     print('Topics:')
# #     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

# # calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

# build doc2author dictionary

# doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

# from gensim.models import atmodel
# doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

# gensim.models.atmodel.construct_author2doc(doc2author)
# # construct mapping from author IDs to document IDs.

# Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
# Returns:	Mapping of authors to document ids.
# Return type:	dict of (str, list of int)

# gensim.models.atmodel.construct_doc2author(corpus, author2doc)
# construct mapping from document IDs to author IDs

# Parameters:	
# corpus (iterable of list of (int, float)) – Corpus in BoW format.
# author2doc (dict of (str, list of int)) – Mapping of authors to documents.
# Returns:	
# Document to Author mapping.

# Return type:	
# dict of (int, list of str)

#### Appendix - Model #1 - Evaluate - holdout set

In [1]:
# appendix
# TODO (Lee) - evaluate on 1k documents **not** used in LDA training
doc_stream = (tokens for _, tokens in iter_wiki('./data/simplewiki-20140623-pages-articles.xml.bz2'))  # generator
test_docs = list(itertools.islice(doc_stream, 8000, 9000))

NameError: name 'iter_wiki' is not defined

#### Appendix - Model #1 - Evaluate - Doc split

In [ ]:
# TODO (Lee) - split each document into two parts, and check that 1) topics of the first half are similar to 
topics of the second 2) halves of different documents are mostly dissimilar:

In [ ]:
# TODO (Lee)
def intra_inter(model, test_docs, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[id2word.doc2bow(tokens[: len(tokens) / 2])] for tokens in test_docs]
    part2 = [model[id2word.doc2bow(tokens[len(tokens) / 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    print(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))

    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
    print("average cosine similarity between 10,000 random parts (lower is better):")    
    print(np.mean([gensim.matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))

In [ ]:
# TODO (Lee)
print("LDA results:")
intra_inter(lda_model, test_docs)

In [ ]:
#### Appendix - Model #1 - Evaluate - Log likelihood

In [ ]:
#### Appendix - Model #1 - Evaluate - Alternate unimplemented workflow

In [ ]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_to_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

In [ ]:
#### Appendix - Model 1 - Inference - Alternate workflows

In [ ]:
# `get_document_topics()` returns topic probability distribution for given document
# topic_dist_675_a = model_lda.get_document_topics(corpus[50])
# pprint(sorted(topic_dist_50_a))

In [ ]:
# topicid = 3
# model_lda.get_topic_terms(topicid, topn=10)

In [ ]:
# text_train[doc_id]
# doc_id = 675
# topic_dist_675_b = sorted(get_topics(corpus[doc_id], k=10)), text_train[doc_id]
# pprint(topic_dist_675_b)

In [ ]:
# From Gensim example - Alternate predict workflow - Create a new corpus, made of previously unseen documents.
# other_texts = [
#      ['computer', 'time', 'graph'],
#      ['survey', 'response', 'eps'],
#      ['human', 'system', 'computer']
#  ]
# other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
# unseen_doc = other_corpus[0]
# vector = lda[unseen_doc]  # get topic probability distribution for a document
# Update the model by incrementally training on the new corpus

In [ ]:
# print keywords in n topics
sorted(model_lda.show_topics(), key=lambda x: x[1])

# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[1])

# print keywords in n topics
sorted(model_l.print_topics(), key=lambda x: x[1])

# print keywords in n topics
sorted(model_1.print_topics(), key=lambda x: x[0])

# show_topic() returns n most important/relevant words, and their weights, that comprise given topic
pprint(model_1.show_topic(1, topn=10))

pprint(model_1.show_topics(num_topics=5, num_words=10))

In [ ]:
#### Appendix - Model #2 - Evaluate - Perplexity
No implementation of log_perplexity method for LDAMallet

In [ ]:
#### Appendix - Train Model #5

In [ ]:
## Appendix - this was alternative partial workflow to create inv:doc mapping in model #5
# uncomment to construct inventor-to-doc mapping as df from nested inventors column in json api response
df_inventors = json_normalize(raw_data_1000, record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.head(3)